In [1]:
!pip install openvino
!pip install onnx
# the torch require onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 48.1 MB/s eta 0:00:00


In [2]:
import datetime
import time


In [3]:

import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from openvino.runtime import Core


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Original model training
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')


# Original model testing
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


accuracy = test_model(model, testloader)
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')





###### The following is openvino part


dummy_input = torch.randn(1, 1, 28, 28)



torch.onnx.export(model, dummy_input, "simple_model.onnx", export_params=True,
                          opset_version=11, do_constant_folding=True,
                          input_names=['input'], output_names=['output'],
                          dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
print("Model has been converted to ONNX format and saved as simple_model.onnx")




ie = Core()

model_path = "simple_model.onnx"
ovmodel = ie.read_model(model=model_path)

# Compile the model for a specific device
compiled_ovmodel = ie.compile_model(model=ovmodel, device_name="CPU")


input_layer = compiled_ovmodel.input(0)
output_layer = compiled_ovmodel.output(0)

correct = 0
total = 0

# Open vino test
for images, labels in testloader:
    images = images.numpy()

    for i in range(images.shape[0]):
        image = images[i]
        image = image.reshape((1, 1, 28, 28))
        results = compiled_ovmodel([image])[output_layer]

        pred = np.argmax(results, axis=1)
        correct += (pred == labels[i].numpy()).sum().item()
        total += labels.size(0)


print(f'Accuracy on the MNIST test set: {accuracy:.4f}')


100%|██████████| 9912422/9912422 [00:00<00:00, 113712902.04it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 99782284.86it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32861303.72it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 19479068.27it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch 1, Loss: 0.8300585974571801
Epoch 2, Loss: 0.1644498827174179
Epoch 3, Loss: 0.1075042290047113
Epoch 4, Loss: 0.08441964117214401
Epoch 5, Loss: 0.07054655564907215
Epoch 6, Loss: 0.06140543161325458
Epoch 7, Loss: 0.05459723339008012
Epoch 8, Loss: 0.048943941540736106
Epoch 9, Loss: 0.044910253075980094
Epoch 10, Loss: 0.04206318167948734
Finished Training
Accuracy of the model on the 10000 test images: 98.77 %
Model has been converted to ONNX format and saved as simple_model.onnx
Accuracy on the MNIST test set: 98.7700


In [4]:
# The follwing is evaluation of time of openvino comparing with original model.
# The timer only calculate the exact time of model runing time for testdata (other like compiling and calculating performance won't be included)
# note that the torch model expect to run the testdata with batch datasets. In this evaluation, torch model only take input 1 data for each loop.





ie = Core()


model_path = "simple_model.onnx"
ovmodel = ie.read_model(model=model_path)


compiled_ovmodel = ie.compile_model(model=ovmodel, device_name="CPU")


input_layer = compiled_ovmodel.input(0)
output_layer = compiled_ovmodel.output(0)

correct = 0
correcto = 0
t_c=0
to_c=0

total = 0
for images, labels in testloader:
    images = images.numpy()

    for i in range(images.shape[0]):

        image = images[i]
        image = image.reshape((1, 1, 28, 28))

        to_a=time.time()
        results = compiled_ovmodel([image])[output_layer]
        to_b=time.time()
        to_c=to_c+to_b-to_a

        t_a=time.time()
        outputs = model(torch.from_numpy(image))
        t_b=time.time()
        t_c=t_c+t_b-t_a


        _, predicted = torch.max(outputs.data, 1)
        correcto += (predicted == labels[i]).sum().item()


        pred = np.argmax(results, axis=1)
        correct += (pred == labels[i].numpy()).item()
    total += labels.size(0)


print("ori_model accuracy:",correcto/total)
print("openvino_model accuracy:",correct/total)
print("ori_model:",t_c,"openvino_model:",to_c)

ori_model accuracy: 0.9877
openvino_model accuracy: 0.9877
ori_model: 10.502442836761475 openvino_model: 5.389642953872681


In [5]:
# the following is the evaluation of total time for runing the whole testdata with openvino(including calculating performance)


model_path = "simple_model.onnx"
ovmodel = ie.read_model(model=model_path)


compiled_model = ie.compile_model(model=ovmodel, device_name="CPU")


input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

correct = 0
correcto = 0

total = 0


a = time.time()
for images, labels in testloader:
    images = images.numpy()

    for i in range(images.shape[0]):

        image = images[i]
        image = image.reshape((1, 1, 28, 28))
        results = compiled_model([image])[output_layer]


        pred = np.argmax(results, axis=1)
        correct += (pred == labels[i].numpy()).item()
    total += labels.size(0)

b = time.time()

print(correct/total)
print("time: ",b-a)


0.9877
time:  5.713577747344971


In [6]:
# the following is the evaluation of total time for runing the whole testdata with original model(including calculating performance)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


a = time.time()
accuracy = test_model(model, testloader)
b = time.time()
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')
print("time: ",b-a)

Accuracy of the model on the 10000 test images: 98.77 %
time:  4.585016489028931


In [7]:
# the following is the evaluation of total time for runing the whole testdata with original model(including calculating performance)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    k_c=0
    with torch.no_grad():
        for data in testloader:
            images, labels = data

            k_a=time.time()
            outputs = model(images)
            k_b=time.time()
            k_c=k_c+k_b-k_a

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print("ptime: ",k_c)
    return accuracy

a = time.time()
accuracy = test_model(model, testloader)
b = time.time()
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')
print("time: ",b-a)


ptime:  2.002572536468506
Accuracy of the model on the 10000 test images: 98.77 %
time:  3.9503231048583984
